In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

from nems import Model
from nems.tf.model import build_model
from nems.layers import WeightChannels

n_chans = 4
wc = WeightChannels(shape=(18, n_chans), name='wc')
wc.as_tensorflow_layer()()

In [ ]:
x = np.random.rand(10, 1000, 18)  # 10 stimuli (as spectrogram) 1000 bins each
y = np.random.rand(10, 1000, n_chans)   # 10 corresponding responses
data = {'input': x}#, 'target': y}

nems_model = Model(wc)
wc_tf = nems_model['wc'].as_tensorflow_layer()()
# Check that input shape gets evaluated correctly
wc_tf.call(keras.Input(shape=(1000,18), name='test', dtype='float32'))

In [ ]:
tf_layers = [layer.as_tensorflow_layer()() for layer in nems_model.layers]
tf_model = build_model(nems_model, tf_layers, data,
                       eval_kwargs={'input_name': 'input'})

In [ ]:
# TODO: move NEMS Model.__repr__ to Model.summary() instead of Model.__str__?
#       (or maybe both, but still switch __repr__ to compact version)
tf_model.summary()  

In [ ]:
tf_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss={
        'wc': keras.losses.MeanSquaredError()
    }
)

In [ ]:
tf_model.fit(
    data, {'wc': y}
)

In [ ]:
tf_model.predict(x).shape

In [ ]:
tf_model.layers[1].weights_to_values()